In [162]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from rac.pred_models import CustomTensorDataset, ACCNet
import numpy as np
np.random.seed(42)

In [2]:
transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])

batch_size = 256

trainset = torchvision.datasets.CIFAR10(root='../datasets/cifar10_original_data', train=True,
                                        download=True, transform=transform)
#trainset.data = trainset.data
#trainset.targets = trainset.targets
X_train = trainset.data
y_train = trainset.targets
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=8)

testset = torchvision.datasets.CIFAR10(root='../datasets/cifar10_original_data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1024, num_workers=8)

X_test = testset.data
y_test = testset.targets

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    

Files already downloaded and verified
Files already downloaded and verified


In [3]:
import numpy as np
X_train = np.load("X_train_cifar.npy")
y_train = np.load("Y_train_cifar.npy")
X_test = np.load("X_test_cifar.npy")
y_test = np.load("Y_test_cifar.npy")

In [870]:
from sklearn.datasets import make_classification
X, Y = make_classification(
    n_samples=500,
    n_features=10,
    n_informative=10,
    n_redundant=0,
    n_repeated=0,
    n_classes=10,
    n_clusters_per_class=1,
    weights=None,
    flip_y=0,
    class_sep=1.0,
    hypercube=True,
    shift=0.0,
    scale=1.0,
    shuffle=True,
    random_state=42)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [86]:
#X = np.load("../datasets/cifar10_data/cifar10_embedding.npy")
#Y = np.load("../datasets/cifar10_data/cifar10_labels.npy")

In [87]:
#from sklearn import preprocessing
#X = preprocessing.StandardScaler().fit_transform(X)
#from sklearn.decomposition import PCA
#pca = PCA(n_components=100)
#X = pca.fit_transform(X)

In [88]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [89]:
#X.shape

In [871]:
def clustering_from_clustering_solution(clustering_solution):
    num_clusters = np.max(clustering_solution) + 1
    clustering = [[] for _ in range(num_clusters)]
    for i in range(len(clustering_solution)):
        clustering[clustering_solution[i]].append(i)
    return clustering, num_clusters

def sim_matrix_from_clustering(clustering, N):
    pairwise_similarities = -np.ones((N, N))
    for cind in clustering:
        pairwise_similarities[np.ix_(cind, cind)] = 1
    return pairwise_similarities

In [979]:
train_sol = clustering_from_clustering_solution(y_train[:10000])
train_sim_matrix = sim_matrix_from_clustering(train_sol[0], len(y_train[:10000]))

test_sol = clustering_from_clustering_solution(y_test[:10000])
test_sim_matrix = sim_matrix_from_clustering(test_sol[0], len(y_test[:10000]))

In [980]:
train_sim_matrix.shape

(400, 400)

In [981]:
def get_pairs(prop_pos, prop_neg, sim_matrix, data):
    N = sim_matrix.shape[0]
    lower_triangle_indices = np.tril_indices(N, -1)
    ind_pos = np.where(sim_matrix[lower_triangle_indices] == 1)[0]
    ind_neg = np.where(sim_matrix[lower_triangle_indices] == -1)[0]
    num_pos = int(len(ind_pos)*prop_pos)
    num_neg = int(len(ind_neg)*prop_neg)
    print("num_pos: ", num_pos)
    print("num_neg: ", num_neg)
    ind_pos = np.random.choice(ind_pos, num_pos)
    ind_neg = np.random.choice(ind_neg, num_neg)
    if num_pos < num_neg:
        indices = np.concatenate([ind_neg, ind_pos])
    else:
        indices = np.concatenate([ind_pos, ind_neg])
    ind1, ind2 = lower_triangle_indices[0][indices], lower_triangle_indices[1][indices]
    x1 = data[ind1]
    x2 = data[ind2]
    y = sim_matrix[ind1, ind2]
    lab1 = np.where(y >= 0)
    lab2 = np.where(y < 0)
    y[lab1] = 1.0
    y[lab2] = 0.0
    return x1, x2, y

In [984]:
x1_train, x2_train, y_train_pairs = get_pairs(0.05, 0.056, train_sim_matrix, X_train)
x1_test, x2_test, y_test_pairs = get_pairs(1, 1, test_sim_matrix, X_test)

num_pos:  390
num_neg:  4030
num_pos:  468
num_neg:  4482


In [742]:
cifar_training_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

cifar_test_transform = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = CustomTensorDataset(x1_train, x2_train, y_train_pairs, train=True, transform=cifar_training_transform)
test_dataset = CustomTensorDataset(x1_test, x2_test, y_test_pairs, transform=cifar_test_transform)  

In [985]:
train_dataset = CustomTensorDataset(torch.Tensor(x1_train), torch.Tensor(x2_train), torch.Tensor(y_train_pairs), train=True, transform=None)
test_dataset = CustomTensorDataset(torch.Tensor(x1_test), torch.Tensor(x2_test), torch.Tensor(y_test_pairs), transform=None)

In [986]:
batch_size = 16
#class_sample_count = [10, 1, 20, 3, 4] # dataset has 10 class-1 samples, 1 class-2 samples, etc.
class_sample_count = np.unique(y_train_pairs, return_counts=True)[1].tolist()
#print(class_sample_count)
weights = 1/torch.Tensor(class_sample_count)
weights = weights[y_train_pairs]
#print(weights)
#sampler = torch.utils.data.sampler.WeightedRandomSampler(weights=weights, num_samples=len(weights), replacement=True)
#train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=1024)
#trainloader = data_utils.DataLoader(train_dataset, batch_size = batch_size, shuffle=True, sampler = sampler)

In [951]:
#for x1, x2, y in train_loader:
    #print(np.unique(y, return_counts=True))

In [987]:
len(train_dataset)

8060

In [988]:
criterion = nn.BCEWithLogitsLoss()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [989]:
device

device(type='cuda', index=0)

In [1031]:
net = ACCNet(base_net="three_layer_net", siamese=True, input_dim=10).to(device)

In [1032]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001, weight_decay=0.00005)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.0005, momentum=0.9)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.0005, momentum=0.9, weight_decay=5e-4)
print("training...")
print(len(train_dataset))
for epoch in range(150):  # loop over the dataset multiple times
    print(epoch)
    running_loss = 0.0
    step = 0
    for i, data in enumerate(train_loader, 0):
        #print(np.unique(data[2], return_counts=True))
        #print(data[2])
        # get the inputs; data is a list of [inputs, labels]
        x1, x2, labels = data[0].to(device), data[1].to(device), data[2].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(x1, x2)
        outputs = outputs.reshape((outputs.shape[0]))
        #labels = labels.reshape((labels.shape[0], 1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        step += 1
    print("loss: ", running_loss/step)
    step = 0
    running_loss = 0.0

training...
8060
0
loss:  0.6462496510810323
1
loss:  0.44796360712794087
2
loss:  0.24222973340176165
3
loss:  0.140749839688873
4
loss:  0.07890172631157348
5
loss:  0.04969435615907608
6
loss:  0.041653065477663724
7
loss:  0.031871668354676606
8
loss:  0.025985582295626437
9
loss:  0.025823563280215556
10
loss:  0.014000800702065351
11
loss:  0.0067112654342775975
12
loss:  0.012852551223774739
13
loss:  0.008469418352274622
14


In [992]:
from scipy.stats import entropy as scipy_entropy
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
preds = []
entropys = []
with torch.no_grad():
    for data in test_loader:
        x1, x2, labels = data[0].to(device), data[1].to(device), data[2].to(device)
        # calculate outputs by running images through the network
        outputs = net(x1, x2)
        pred = nn.Sigmoid()(outputs)
        #print(pred)
        pred = pred.cpu().numpy()
        pred = pred.reshape(pred.shape[0])
        prob = [1-pred, pred]
        entropy = scipy_entropy(prob)
        pred[pred >= 0.5] = 1
        pred[pred < 0.5] = 0
        #print(torch.max(pred.data, 1)[1])
        # the class with the highest energy is what we choose as prediction
        #_, predicted = torch.max(pred.data, 1)
        preds.extend(pred.tolist())
        entropys.extend(entropy.tolist())


In [1024]:
preds = np.array(preds)
entropys = np.array(entropys)

inds = np.where(entropys < 0.001)[0]
preds_new = preds[inds]
y_test_pairs_new = y_test_pairs[inds]


In [1025]:
np.unique(preds_new, return_counts=True)

(array([0.]), array([2154], dtype=int64))

In [1026]:
#preds

In [1027]:
len(y_test_pairs)


4950

In [1028]:
y_test_pairs

array([0., 0., 0., ..., 1., 1., 1.])

In [1029]:
np.unique(y_test_pairs, return_counts=True)

(array([0., 1.]), array([4482,  468], dtype=int64))

In [1030]:
from sklearn.metrics import classification_report
print(classification_report(y_test_pairs_new, preds_new))

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      2136
         1.0       0.00      0.00      0.00        18

    accuracy                           0.99      2154
   macro avg       0.50      0.50      0.50      2154
weighted avg       0.98      0.99      0.99      2154



c:\Users\Linus\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Linus\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Linus\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
